# DQN

DQN (Deep Q-Network) is a method to implement Q learning - an off policy value based method - using deep neural networks. Q learning when using non linear function approximators can be very unstable and data inefficient so DQN addresses these problems. [DQN Paper](https://arxiv.org/pdf/1312.5602.pdf)

The following is DQN pseudocode

![Pseudocode](https://lilianweng.github.io/lil-log/assets/images/DQN_algorithm.png)

## DQN Network

The DQN takes in the state input and outputs Q values for all possible actions.

DQNs can only be used with discrete policies.

In [68]:
from torch import nn
import torch.nn as nn
import torch.nn.functional as F
import torch

class DQN(nn.Module):

    def __init__(self, input_size, hidden_size, nb_actions) -> None:
        super(DQN, self).__init__()
        self.DQN = nn.Sequential(nn.Linear(input_size, hidden_size),
                                        nn.LeakyReLU(),
                                        nn.Linear(hidden_size, hidden_size),
                                        nn.LeakyReLU(),
                                        nn.Linear(hidden_size, nb_actions),
                                        nn.LeakyReLU())
        
    def forward(self,obs):
        return self.DQN(obs)

## Replay Memory

DQNs make use of experience replay memory - this stores previous transitions to allow for data efficient and non correlated learning to take place.

In [69]:
from collections import namedtuple, deque
import random
import numpy as np

Transition = namedtuple(
    "Transition", ("state", "action", "reward", "next_state", "done"))

class Replay_Memory:

    def __init__(self, size):
        self.memory = deque([], maxlen=size)

    def push(self, *args):
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        if batch_size > len(self.memory):
            return None
        return random.sample(self.memory, batch_size)


## Agent 
The following is the agent class:

In [70]:
from torch.distributions import Categorical, Normal
from torch.optim import AdamW
import torch

class DQN_Agent:

    def __init__(self,input_size,hidden_size,nb_actions,discount_factor=0.9,learning_rate=0.0001,epsilon_start=0.9,epsilon_end=0.05,epsilon_anneal_over_steps=10000,batch_size=256,update_frequency = 1000,replay_memory_size=100000):
        
        # Create the online network that the agent uses to select actions
        self.online_net = DQN(input_size,hidden_size,nb_actions)

        # Create the target network that the agent uses in it's updates
        self.target_net = DQN(input_size,hidden_size,nb_actions)

        self.num_actions = nb_actions

        # Set the target net's initial weights to equal the online net
        self.target_net.load_state_dict(self.online_net.state_dict())

        self.gamma = discount_factor
        
        # User Huber Loss function - others can be use such as MSE
        self.loss_function = torch.nn.SmoothL1Loss()

        # DQN optimizer
        self.optimizer = AdamW(self.online_net.parameters(),learning_rate)
        
        # Set initial and final epsilon values for epsilon greedy and choose the number of environment steps to anneal over
        self.epsilon_end = epsilon_end
        self.epsilon_start = epsilon_start
        self.epsilon_anneal_over_steps = epsilon_anneal_over_steps
        self.step_no = 0

        # Create Replay Memory and assign batch_size
        self.replay = Replay_Memory(replay_memory_size)
        self.batch_size = batch_size
        
        # Set update Frequency
        self.update_frequency = update_frequency
        
        # Best Weights for Eval
        self.best_weights = self.online_net.state_dict()

    def get_epsilon(self):
        """
        Get current epsilon value according to agents total step number in the environment
        """
        eps = self.epsilon_end
        if self.step_no < self.epsilon_anneal_over_steps:
            eps = self.epsilon_start - self.step_no * \
                ((self.epsilon_start - self.epsilon_end) /
                 self.epsilon_anneal_over_steps)
        return eps
        
    def act(self,obs):
        # Increment global step count
        self.step_no+=1

        if np.random.uniform() > self.get_epsilon():
            # Dont store gradients when acting in the environment
            with torch.no_grad():
                return torch.argmax(self.online_net(obs),dim=-1).view(1)
        else:
            return torch.tensor([random.randrange(self.num_actions)], dtype=torch.long)

    # store experience in replay memory
    def cache(self, state, action, reward, next_state, done):
        self.replay.push(state, action, reward, next_state, done)

    def train(self):
        self.online_net.train()
        self.target_net.train()
    
    def eval(self):
        self.online_net.load_state_dict(self.best_weights)
        self.online_net.eval()
        self.target_net.eval()

    def update_model(self):

        # Get minibatch of data from experience buffer
        batch = self.replay.sample(self.batch_size)

        # If memory doesnt have enough transitions
        if batch == None:
            return

        # Format batch to get a tensor of states, actions, rewards, next states and done booleans
        batch_tuple = Transition(*zip(*batch))
        state = torch.stack(batch_tuple.state)
        action = torch.stack(batch_tuple.action)
        reward = torch.stack(batch_tuple.reward)
        next_state = torch.stack(batch_tuple.next_state)
        done = torch.stack(batch_tuple.done)

        self.optimizer.zero_grad()

        # Get the Q values of the online nets current state and actions
        Q_Values = self.online_net(state).gather(1, action).squeeze()

        # Get the max Q values of the target nets next state
        Q_Targets = reward + (1 - done.float()) * self.gamma *self.target_net(next_state).max(dim=-1)[0].detach()
        
        # Calculate loss
        loss = self.loss_function(Q_Values, Q_Targets)

        # Calculate the gradients
        loss.backward()

        # Perform optimization step
        self.optimizer.step()

        return loss.item()
    

    def update_target(self):
        """
        Update the target nets weights
        """
        self.target_net.load_state_dict(self.online_net.state_dict())

    def update_best_weights(self):
        self.best_weights = self.online_net.state_dict()


## Training Loop
The last thing needed is the training loop.

In [71]:
from collections import deque
import gym
from numpy import double
import torch
import argparse

def train(agent, env, num_episodes, num_steps, log_every=20, render_from=1000):
    """
    :param agent: the agent to be trained.
    :param env: the gym environment.
    :param num_episodes: the number of episodes to train.
    :param num_steps: the max number of steps possible per episode.
    :param log_every: The frequency of logging. Default logs every 20 episodes.
    :param render_from: The episode number to start rendering to screen to allow one to view agent. Rendering significantly slows down training.
    """
    
    # Running Average Reward Memory
    running_avg_reward = deque(maxlen=100)
    best_running_avg = 0
    agent.train()
    for episode in range(1,num_episodes+1):
        # Starting state observation
        obs = torch.tensor(env.reset(),dtype=torch.float32)
        reward_total = 0
        for step in range(num_steps):
            if episode>render_from:
                env.render()

            # return chosen action
            action = agent.act(obs)
            
            # Take a step in the environment with the action drawn
            next_obs , reward, done, info = env.step(action.item())
            
            # Just for logging
            reward_total+=reward

            # change next state into tensor for update
            next_obs = torch.tensor(next_obs,dtype=torch.float32)
            
            # change reward into tensor for update
            reward = torch.tensor(
                reward, dtype=torch.float32)

            # Store transition in replay memory
            agent.cache(obs,action,reward,next_obs,torch.tensor(done))

            # Perform update
            agent.update_model()
            
            # If the number of steps has elapsed then perform update
            if agent.step_no % agent.update_frequency == 0:
                agent.update_target()

            # Set the current state to the next state
            obs = next_obs
            
            # if done then log and break
            if done:
                if episode % log_every ==0 and len(running_avg_reward)>0:
                    running_avg = sum(running_avg_reward)/len(running_avg_reward)
                    if running_avg > best_running_avg:
                        agent.update_best_weights()
                        
                    print("Episode {0:4d} finished after {1:4d} timesteps with a total reward of {2:3.1f} | Running Average: {3:3.2f}".format(episode,step+1,reward_total,running_avg))
                break
        running_avg_reward.append(reward_total)
        

    env.close()

In [72]:

def eval(agent, env, num_episodes, num_steps, log_every=20):
    """
    :param agent: the agent to be trained.
    :param env: the gym environment.
    :param num_episodes: the number of episodes to eval.
    :param num_steps: the max number of steps possible per episode.
    :param log_every: The frequency of logging. Default logs every 20 episodes.
    """
    agent.eval()
    # Running Average Reward Memory
    running_avg_reward = deque(maxlen=100)
    for episode in range(1,num_episodes+1):
        # Starting state observation
        obs = torch.tensor(env.reset(),dtype=torch.float32)
        reward_total = 0
        for step in range(num_steps):
           
            env.render()
            # return chosen action
            action = agent.act(obs)
            
            # Take a step in the environment with the action drawn
            next_obs , reward, done, info = env.step(action.item())
            
            # Just for logging
            reward_total+=reward

            # change next state into tensor for update
            next_obs = torch.tensor(next_obs,dtype=torch.float32)
            
            # Set the current state to the next state
            obs = next_obs
           
            # if done then log and break
            if done:
                if episode % log_every ==0 and len(running_avg_reward)>0:
                    running_avg = sum(running_avg_reward)/len(running_avg_reward)
                    print("Episode {0:4d} finished after {1:4d} timesteps with a total reward of {2:3.1f} | Running Average: {3:3.2f}".format(episode,step+1,reward_total,running_avg))
                break
        running_avg_reward.append(reward_total)
    env.close()

## Time to Train

In [73]:
env = gym.make("CartPole-v0")


nb_actions = env.action_space.n

# Hyper parameters
HIDDEN_SIZE = 512
GAMMA = 0.99
LEARNING_RATE = 0.0005
EPISODES_TO_TRAIN = 700
EPISODES_TO_EVAL = 20
MAX_STEPS = 201
LOG_EVERY = 20
RENDER_FROM_EP = 2000
EPSILON_START=0.9
EPSILON_END=0.01
EPSILON_ANNEAL_OVER_STEPS=10000
BATCH_SIZE=512
UPDATE_FREQUENCY = 1500
REPLAY_MEMORY_SIZE=500000


agent = DQN_Agent(env.observation_space.shape[0],HIDDEN_SIZE,nb_actions,GAMMA,LEARNING_RATE,EPSILON_START,EPSILON_END,EPSILON_ANNEAL_OVER_STEPS,BATCH_SIZE,UPDATE_FREQUENCY,REPLAY_MEMORY_SIZE)

print("Training...")

train(agent,env,EPISODES_TO_TRAIN,MAX_STEPS,LOG_EVERY,RENDER_FROM_EP)

print("Evaluating...")

eval(agent,env,EPISODES_TO_EVAL,MAX_STEPS,log_every=1)

Training...
Episode   20 finished after   28 timesteps with a total reward of 28.0 | Running Average: 19.74
Episode   40 finished after   19 timesteps with a total reward of 19.0 | Running Average: 22.18
Episode   60 finished after   12 timesteps with a total reward of 12.0 | Running Average: 21.03
Episode   80 finished after   21 timesteps with a total reward of 21.0 | Running Average: 20.77
Episode  100 finished after   35 timesteps with a total reward of 35.0 | Running Average: 20.73
Episode  120 finished after   30 timesteps with a total reward of 30.0 | Running Average: 22.35
Episode  140 finished after   87 timesteps with a total reward of 87.0 | Running Average: 22.76
Episode  160 finished after   16 timesteps with a total reward of 16.0 | Running Average: 25.82
Episode  180 finished after   38 timesteps with a total reward of 38.0 | Running Average: 27.85
Episode  200 finished after   18 timesteps with a total reward of 18.0 | Running Average: 29.04
Episode  220 finished after 